# Regularization, Dropout, Early Stopping, Batch Normalization

## Imports

In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torchmetrics
import wandb

from tqdm.auto import tqdm
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from torchsummary import summary


## Device and Deterministic 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
# Psudecode
# dataset fucntion
# data loader
# Model 
# make fucntion - loads everything for use 
# Model pipeline 
# configs
# train and batch_train
# 


# Wandb login

In [14]:
wandb.init(
    project= 'flower102-multirun',
    name = 'ModelV3' ,
    job_type = 'model-training',
)


wandb: Currently logged in as: shushanksingh310 (shushankai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
config = dict(
    img_heigth = 224,
    img_width = 224, 
    in_channels = 3, 
    batch_size = 16,
    epoch = 10,
    hidden_units = 64,
    in_features = int(224 * 224 * 3),
    out_features = 102,
    learning_rate = 1e-3
)


In [ ]:
config


{'img_heigth': 224,
 'img_width': 224,
 'in_channels': 3,
 'batch_size': 16,
 'epoch': 10,
 'hidden_units': 64,
 'in_features': 150528,
 'out_features': 102,
 'learning_rate': 0.001}

# Dataset and Dataloader 


In [16]:
def get_data(split, transform, subset=False, slice=5):
    """"""
    path = "Data/train/"
    
    if split == 'val':
        path = 'Data/val/'
    
    if split == 'test':
        path = "Data/test/"
        
    full_dataset = torchvision.datasets.Flower102(
        root = path, 
        download = True, 
        transform = transform,
        split= split
    )
    
    if subset:
        sub_dataset = torch.utils.data.Subset(
            dataset = full_dataset,
            indices = range(0, len(full_dataset), slice)
        )
        return sub_dataset
    return full_dataset
    
    


In [11]:
def transform(resize):
    return transforms([
        transforms.Resize(resize),
        transforms.ToTensor()
    ])



In [ ]:
def make_loader(dataset, batch_size, shuffle):
    loader = DataLoader(
        dataset = dataset,
        batch_size = batch_size, 
        shuffle = shuffle,
        num_worker = 2,
        pin_memory = True
    )
    return loader


## Model

In [3]:
class ModelV3(nn.Module):
    def __init__(self, in_features, out_features, hidden_units):
        super(ModelV3, self).__init__()
        self.flatten = nn.Flatten()
        self.layer1 = nn.Linear(
            in_features = in_features, 
            out_features = hidden_units,
            bias = True
        )
        self.layer2 = nn.Linear(
            in_features = hidden_units, 
            out_features = out_features,
            bias = True
        )
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x


## Train

In [15]:
def train(model, loader, loss, optimizer, config):
    wandb.watch(
        models = model,
        criterion = loss,
        log = 'all',
        log_freq = 10
    )
    
    total_batch = len(loader) * config.epochs
    example_ct = 0
    batch_ct = 0
    
    model.train()
    
    for epoch in tqdm(range(config.epochs)):
        for _, (images, labels) in enumerate(loader):
            loss = train_batch(images, labels, model, optimizer, loss)
            example_ct += 1
            batch_ct += 1
            
            if (batch_ct + 1) % 25 == 0:
                train_log(loss, example_ct, epoch)
                

def train_batch(images, labels, model, optimizer, loss_fn):
    images, labels = images.to(device) , labels.to(device)
    
    logits = model(images)
    batch_loss = loss_fn(logits, labels)
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()

    return batch_loss
    
    
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    print(f"Loss after {str(example_ct).zfill(5)} examples: {loss:.3f}")
    


In [17]:
def make(config):
    model = ModelV3(
        
    )
    flower_transform = transform(resize= [224, 224])
    train_dataset = get_data(
        split= 'train',
        transform= flower_transform,
        subset= False,
    )
    val_dataset = get_data(
        split= 'val',
        transform= flower_transform,
        subset= False
    )
    
    train_loader , val_loader = make_loader(
        dataset = train_dataset, batch_size= config.batch_size, shuffle= True
    ), 
    make_loader(
        dataset= val_dataset, batch_size= config.batch_size, shuffle= True
    )
    

In [18]:
model = ModelV3(
    in_features= 150528,
    out_features= 102, 
    hidden_units= 64
)
model


ModelV3(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layer1): Linear(in_features=150528, out_features=64, bias=True)
  (layer2): Linear(in_features=64, out_features=102, bias=True)
  (relu): ReLU()
)